In [1]:
# Import helper plotting functions
# If you want to see the code, you can access it in here: 
# https://github.com/artoriusss/blockchain-fraud-analysis/blob/main/helpers.py
from helpers import * 

## **0. Data loading & initial check**

In [2]:
INPUT_CSV = "data/test_dataset.csv"
df = pd.read_csv(INPUT_CSV)

To get a high-level understanding of the data, let's print out some basic information:

In [3]:
print(f"Number of rows in a dataframe: {df.shape[0]}\n")
df.info()

Number of rows in a dataframe: 3923

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3923 entries, 0 to 3922
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   address                3923 non-null   object
 1   chain_id               3923 non-null   int64 
 2   bytecode               3923 non-null   object
 3   opcode                 3923 non-null   object
 4   malicious_category_l1  550 non-null    object
 5   malicious_category_l2  550 non-null    object
 6   data_source            3923 non-null   object
 7   man_validated          3791 non-null   object
 8   man_validated_by       944 non-null    object
 9   comment                1004 non-null   object
 10  label                  3923 non-null   object
dtypes: int64(1), object(10)
memory usage: 337.3+ KB


What we can tell from this print out? 

First of all, the number of non-empty records varies across different columns. In case of features like `man_validated_by` or `comment` the presence of missing values seems natural, since these are likely dependent on whether a particular record was validated by someone and if not, then we won't have any information there. 

However, the two fields representing malicious categories are more interesting. Not only because we have the smallest proportions of present values for them ($\approx 14\%$ of the total records number) but also because they are good candidates for our choice of a target varible (feature that we'll try to approximate with our model). So, let's analyse them more deeply.

## **1. Target variable analysis**

We'll start with a simple barplot representing counts of different categories within the features. 

In [4]:
plot_category_counts(df, "malicious_category_l1", "malicious_category_l2")

alt.HConcatChart(...)

From the above chart, one thing immediately comes out as noteworthy: across both features, counts of different categories are distributed identically (exact counts can be revealed with tooltips if you hover over the bars of the charts). 

However, this pattern alone does not yet tell us that level1 `Cyber` category always goes with level2 `Exploit` category, or that `Fraud` always goes with `FinancialScam`. After all, the exact same count may have occurred by accident and these categories may be unrelated at all. So, we need to verify this. 

We could do it "manually", by simply counting cases of overlapping for each category. But we can use a simpler approach: encode categorical variables numerically and calculate the correlation matrix. This way, we'll see the entire picture all at once. 

In [5]:
l1_dummies = pd.get_dummies(df["malicious_category_l1"], prefix="L1") 
label_dummies = pd.get_dummies(df["malicious_category_l2"], prefix="L2")
malicious_categories = pd.concat([l1_dummies, label_dummies], axis=1)

plot_correlation_matrix(
    df=malicious_categories, 
    title='Correlation Matrix of Level1 and Level2 Malicious Categories'
)

alt.LayerChart(...)

From the above chart, we can observe that categories from the 1st and the 2nd levels of malicious categories are *perfectly correlated*. 

In other words:

- All observations labelled on 1st level as belonging to the `Cyber` category are *always* labelled as belonging to the `Exploit` category on the 2nd level; 
- All observations labelled on 1st level as belonging to the `Fraud` category are *always* labelled as belonging to the `FinancialScam` category on the 2nd level;  

To put it even more simply, the two levels of malicious category labels do not give us any additional information about about our dataset. We can safely drop either the first or the second level feature without losing any valuable knowledge about the data. Moreover, this drop will also be necessary on later stages of modelling, since perfectly correlated training data itroduces *multicollinearity* that can strongly degrade the performance of our model.

For this reason, we will move on with only `malicious_category_l1` column (again, this choice is arbitrary, we could keep `l2`).

In [6]:
df.drop(columns=["malicious_category_l2"], inplace=True)

However, apart from malicious category, we have another column that we can choose as a target variable: a smart contract `label` column. It is important to analyse it as well.

In [7]:
label_dummies = pd.get_dummies(df["label"], prefix="Label")
malicious_categories = pd.concat([l1_dummies, label_dummies], axis=1)

plot_correlation_matrix(
    df=malicious_categories, 
    title='Correlation Matrix of Level1 Malicious Categories and Smart Contract Labels'
)

alt.LayerChart(...)

We can again observe a very similar pattern: level1 malicious `Cyber` category is perfectly correlated with `Exploit` label. The same is true for category `Fraud` and `FinancialScam` label. However, in this case we can also note that the correlation coefficient is very close to $1$ but is not exactly $1$. 

This signifies that there are row(s) where the `Fraud` malicious category does not correspond to `FinancialScam` label, but some other label. Let's verify if this is the case. 

In [8]:
selected_rows = df[
    (df["malicious_category_l1"] == "Fraud") & 
    (df["label"] == "Exploit")
]  # Check if there are any frauds that are labelled as exploits

selected_rows[["chain_id", "malicious_category_l1", "label"]]

,chain_id,malicious_category_l1,label


In [9]:
selected_rows = df[
    (df["malicious_category_l1"] == "Fraud") & 
    (df["label"] == "Clean")
]  # Check if there are any frauds that are labelled as clean 

selected_rows[["chain_id", "malicious_category_l1", "label"]]

,chain_id,malicious_category_l1,label
2242,56,Fraud,Clean


Indeed, we see that there's one record where malicious category does not correspond to the label assigned by smart contract. Clearly, the two categories `Fraud` and `Clean` are mutually exclusive and one of the columns must hold an incorrect value, but we don't know for sure which exactly does. 

So, as we saw above, because the two features `malicious_category_l1` and `label` resemble (almost) perfect correlation, it's better to drop one of them as before. But in this case, the columns carry contradicting information, so we need to make a choice about which column to keep and which one to drop. 

In [10]:
selected_rows[["chain_id", "data_source", "man_validated", "man_validated_by", "comment"]]

,chain_id,data_source,man_validated,man_validated_by,comment
2242,56,Public_Historical_Data,NaN,NaN,"reported by HashDit, ponzi"


The record with contradicting labels comes from public historical data and we cannot trust it for sure, so it may be safer to treat smart contract labelled data as more credible in this case. That's why we'll drop the `malicious_category_l1` column and move on with `label` only. Moreover, we will use it as a target variable, since we eliminated other candidates during our analysis.

In [11]:
df.drop(columns=["malicious_category_l1"], inplace=True)

## **2. Duplicates & missing values**

Let's now check our dataset on the presence of duplicates. Because each smart contract deployed on blockchain has its own unique address, it makes sense to focus on this column.

In [12]:
duplicate_rows = df[df.duplicated(subset=["address"])]
print(f"Number of duplicated rows: {duplicate_rows.shape[0]}")
duplicate_rows.value_counts("label")

Number of duplicated rows: 16


label
Clean      13
Exploit     3
Name: count, dtype: int64

As we can see, most of the duplicates belong to clean transactions. However, we still have 3 duplicated rows of fraudulent transactions labelled as `Exploit`. While this may not sound like a lot, we should remember that our dataset is rather imbalanced, meaning that fraudulent transactions are underrepresented compared to the clean ones. For that reason, we may keep these 3 duplicates belonging to malicious transactions and drop only clean ones. 

In [13]:
df = df[~(df.duplicated(subset=["address"]) & (df["label"] == "Clean"))]

## **3. Analysis, Feature engineering**

In this section, our goal is to explore relevant patterns and relations in the data. This step is crucial, as it will lay foundation for the final preprocessed training dataset and modelling.

Let's start by exploring some relations of our target variable with other available features.

### **3.1 `data_source` and `chain_id`**

In [14]:
plot_categorical_bar_charts(df, [{'x': "data_source", 'y': "label"}, {'x': "chain_id", 'y': "label"}])

alt.HConcatChart(...)

Let's go through both charts one step at a time:

- On the first one, it is clear that malicious records distribution is heavily dependent on the datasource. For instance, while `White_Lists` category speaks for itself, records obtained from historical data and monitoring are represented largely by fraudulent transactions. All of this gives us very valuable knowledge about the target variable.

- From the second plot, it is also notable that the proportion of fradulent transactions is somewhat correlated with `chain_id` (for instance, more than a half records with id `56` labelled as malicious, but only small percentages are in case of other chain ids).

This tells us that the both analysed variables are important when it comes to their ability to describe the distribution of a target variable.

### **3.2 Comments**

In [15]:
non_null_comments_count = df['comment'].notnull().sum()
print(f"Number of non-null comments: {non_null_comments_count}")

Number of non-null comments: 1003


As we see, comments are present just in one third of the entire dataset. While this may seem like a bad thing because of the insufficient data, *the presence* of the comment can tell us a lot about the target variable. So let's split the dataset into two parts: with comments missing and present and see if there are any interesting patterns here. 

In [16]:
plot_label_distribution_by_comment(df)

alt.HConcatChart(...)

Turns out that a bit less than a half of the record with comments belong to fraudulent transactions! This makes sense, since reviewers usually can often write comments when there's something important to pay attention to, and fraudulent transactions are of great importance for sure.

Again, we can take advantage of this pattern and create a new boolean feture: `commented`. 

(we could also extract some information from the comments (for instance, we could categorise comments as identifying "good" and "bad" properties of transactions), but this is a more advanced task and we'll skip it for now).

In [17]:
df["is_commented"] = df["comment"].notnull()

### **3.3 `opcode`**

Another important feature is `opcode`. To see if it can serve us as a good predictor, we can divide the opcode into atomic parts (split by space) and compare the most frequent sequences of opcodes in clean and malicious transactions.

In [18]:
from collections import Counter

def most_frequent_opcode_sequence(opcode_str: str, n: int = 3) -> str:
    opcodes = opcode_str.split()
    sequences = [' '.join(opcodes[i:i+n]) for i in range(len(opcodes) - n + 1)]
    sequence_counts = Counter(sequences)
    most_common_sequence = sequence_counts.most_common(1)[0][0]
    return most_common_sequence

df['most_frequent_opcodes'] = df['opcode'].apply(most_frequent_opcode_sequence, n=5)

Here's how the dataset looks like after splitting and extracting the most frequent opcode sequences:

In [19]:
df['most_frequent_opcodes'].value_counts().head(10)

most_frequent_opcodes
PUSH1 PUSH1 PUSH1 SHL SUB          1609
PUSH2 JUMPI PUSH1 DUP1 REVERT       517
ISZERO PUSH2 JUMPI PUSH1 DUP1       257
SWAP2 SWAP1 PUSH2 JUMP JUMPDEST     176
PUSH1 DUP3 ADD MSTORE PUSH1         133
POP POP JUMP JUMPDEST PUSH1         120
POP POP POP POP POP                 119
PUSH2 JUMP JUMPDEST PUSH1 MLOAD      99
PUSH2 JUMPI PUSH2 PUSH2 JUMP         81
ISZERO DUP1 ISZERO PUSH2 JUMPI       72
Name: count, dtype: int64

But in this form is impossible to see any patterns. So, what we can do here is to split our dataset into clean and malicious transactions and then compare the most frequent opcode sequences in both groups. To make a reasonable comparison, however, we can't just look at the numbers side by side. Instead, we want to test whether the difference in the number of occurrences of a particular opcode sequence in clean and malicious transactions is statistically significant.

For that, we'll perform a chi-squared test.

In [20]:
from scipy.stats import chi2_contingency

def chi2_contingency_test(counts_df1: pd.Series, counts_df2: pd.Series) -> None:
    combined_index = counts_df1.index.union(counts_df2.index)
    counts_df1 = counts_df1.reindex(combined_index, fill_value=0)
    counts_df2 = counts_df2.reindex(combined_index, fill_value=0)

    contingency_table = pd.DataFrame({
        'df1': counts_df1,
        'df2': counts_df2
    })

    chi2, p, _, _ = chi2_contingency(contingency_table)

    print(f"Chi-squared statistic: {chi2:.4f}")
    print(f"P-value: {p}")

    if p < 0.05:
        print("The difference in distributions is statistically significant.")
    else:
        print("The difference in distributions is not statistically significant.")

In [21]:
df_clean = df[df['label'] == 'Clean'].value_counts("most_frequent_opcodes")
df_malevolent = df[df['label'] != 'Clean'].value_counts("most_frequent_opcodes")

chi2_contingency_test(df_clean, df_malevolent)

Chi-squared statistic: 619.7126
P-value: 1.955353979407871e-54
The difference in distributions is statistically significant.


As we see, the p-value is very close to $0$, which means that the difference in the number of occurrences of the opcode sequences in clean and malicious transactions is statistically significant. This is a very valuable information, as it tells us that the `opcode` feature can be a good predictor of the target variable.

So we'll keep this feature and make use of it. But we'll take advantage of the full `opcode` column, rather than from the sequences we extracted.

In [22]:
df.drop(columns=["most_frequent_opcodes"], inplace=True)

## **4. Final Dataset Preparation**

### **4.1 Feature selection**

As a result of our analysis, apart from other columns that we already dropped, we'll also remove the following ones:

- `address` - because it's unique for each record and does not provide any valuable information 
- `bytecode` - it is encoded in the `opcode` column and cannot be adequately handled becuse of the unreadable format
- `comment` - we already extracted the information from it and created a new feature
- `man_validated` - the column is highly correlated with the `is_commented` column and does not provide any additional information
- `man_validated_by` - it is unreasonable to assume any pattern in the data based on the person who validated the record

In [23]:
df.drop(columns=["address", "bytecode", "comment", "man_validated", "man_validated_by"], inplace=True)
print(f"The following columns have been kept: {list(df.columns)}")

The following columns have been kept: ['chain_id', 'opcode', 'data_source', 'label', 'is_commented']


### **4.3 Preprocessing**

Fianlly, we'll perform the following preprocessing steps:

- One-hot encode the categorical `data_source`, `chain_id` and `is_commented` columns.
- Format the target variable `label` as a numerical value.
- TF-IDF vectorize the `opcode` column, then apply PCA to reduce the dimensionality of the data (for the sake of simplicity, we'll keep only 10 principal components, but this parameter can surely be tweaked to maximise performance and the speed of calculations). 


In [24]:
data_source_dummies = pd.get_dummies(df["data_source"], prefix="src", drop_first=True)
chain_id_dummies = pd.get_dummies(df["chain_id"], prefix="chain_id", drop_first=True)
is_commented_dummy = pd.get_dummies(df["is_commented"], prefix="is_commented", drop_first=True)

df = pd.concat([df, data_source_dummies, chain_id_dummies, is_commented_dummy], axis=1)
df.drop(columns=["data_source", "chain_id", "is_commented"], inplace=True)

In [25]:
df["label"] = df["label"].map({"Clean": 0, "Exploit": 1, "Fraud": 1, "FinancialScam": 2})

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['opcode'])

pca = PCA(n_components=10)
pca_result = pca.fit_transform(tfidf_matrix.toarray())
pca_columns = [f'pca_{i+1}' for i in range(pca_result.shape[1])]
pca_df = pd.DataFrame(pca_result, columns=pca_columns)
df_with_pca = pd.concat([df.reset_index(drop=True), pca_df], axis=1)
df_with_pca.drop(columns=['opcode'], inplace=True)

df_preprocessed = df_with_pca.copy()

As a result, the final preprocessed dataset before training the model will look as follows:

In [27]:
df_preprocessed.head()

,label,src_Monitoring_and_Feedback,src_Public_Historical_Data,src_Random_Data,src_White_Lists,chain_id_10,chain_id_56,chain_id_137,chain_id_42161,chain_id_43114,...,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10
0,0,False,False,False,True,False,False,False,False,False,...,0.182550,-0.073578,0.007320,-0.000756,-0.083341,-0.026792,0.084152,0.070987,0.010037,0.022671
1,0,False,False,False,True,False,False,False,False,False,...,-0.035425,0.113577,0.086179,-0.027716,-0.090990,0.012979,0.080748,0.022929,0.000644,-0.011327
2,0,False,False,False,True,False,False,False,False,False,...,0.096898,0.026547,0.014584,0.008460,0.088735,0.017421,0.011854,-0.029745,0.024360,-0.031277
3,1,False,True,False,False,False,False,False,False,True,...,-0.123460,-0.061437,-0.204400,-0.151068,-0.032918,-0.013231,0.067481,-0.085884,0.014154,0.024943
4,0,False,False,True,False,False,False,True,False,False,...,-0.305750,0.123946,-0.099024,-0.076704,0.011680,0.024308,-0.025485,-0.031155,-0.009663,-0.006640


### **4.4 Considerations aboun model training**

Given the nature of our data, its imbalance and relatively small size of 3.9k records the reasonable choice for the model would be a tree-based model, like Random Forest or Gradient Boosting. These models are known for their ability to handle imbalanced data the scikit-learn implementation supports weight assignment for each class, which can be very useful in our case. Also, they are less prone to overfitting compared to other models, which is also an important property given our dataset size.